<div style="border-left: 6px solid #FF8C42; color:white; padding:20px; border-radius:10px; font-family:Arial, sans-serif; text-align:center; font-size:28px; font-weight:bold;">
  ⚙️ 04 – Hyperparameter Tuning
</div>
